In [2]:
import sys
import gc
#sys.path
sys.path.insert(0, '../')

In [3]:
sys.path

['../',
 'C:\\Users\\user\\Documents\\Accelerate\\CollaborativeProject\\recommendation_system\\collaborative_filtering',
 'C:\\Users\\user\\Anaconda3\\envs\\recommender\\python37.zip',
 'C:\\Users\\user\\Anaconda3\\envs\\recommender\\DLLs',
 'C:\\Users\\user\\Anaconda3\\envs\\recommender\\lib',
 'C:\\Users\\user\\Anaconda3\\envs\\recommender',
 '',
 'C:\\Users\\user\\Anaconda3\\envs\\recommender\\lib\\site-packages',
 'C:\\Users\\user\\Anaconda3\\envs\\recommender\\lib\\site-packages\\win32',
 'C:\\Users\\user\\Anaconda3\\envs\\recommender\\lib\\site-packages\\win32\\lib',
 'C:\\Users\\user\\Anaconda3\\envs\\recommender\\lib\\site-packages\\Pythonwin',
 'C:\\Users\\user\\Anaconda3\\envs\\recommender\\lib\\site-packages\\IPython\\extensions',
 'C:\\Users\\user\\.ipython']

In [4]:
import pandas as pd
from Data_cleaning import get_clean_data
from Data_cleaning import get_merged_data_frame

### Load Data  
we now load the data using Alfred's framework.

In [5]:
df_merged = get_merged_data_frame(user_argv=10, isbn_argv=10, path='../data/')

b'Skipping line 6452: expected 8 fields, saw 9\nSkipping line 43667: expected 8 fields, saw 10\nSkipping line 51751: expected 8 fields, saw 9\n'
b'Skipping line 92038: expected 8 fields, saw 9\nSkipping line 104319: expected 8 fields, saw 9\nSkipping line 121768: expected 8 fields, saw 9\n'
b'Skipping line 144058: expected 8 fields, saw 9\nSkipping line 150789: expected 8 fields, saw 9\nSkipping line 157128: expected 8 fields, saw 9\nSkipping line 180189: expected 8 fields, saw 9\nSkipping line 185738: expected 8 fields, saw 9\n'
b'Skipping line 209388: expected 8 fields, saw 9\nSkipping line 220626: expected 8 fields, saw 9\nSkipping line 227933: expected 8 fields, saw 11\nSkipping line 228957: expected 8 fields, saw 10\nSkipping line 245933: expected 8 fields, saw 9\nSkipping line 251296: expected 8 fields, saw 9\nSkipping line 259941: expected 8 fields, saw 9\nSkipping line 261529: expected 8 fields, saw 9\n'
C:\Users\user\Anaconda3\envs\recommender\lib\site-packages\IPython\core\in

Actually we only need the data from df_ratings. But we merge the data and just deleting most of columns again.
We now end with less rows, because we have some ratings without corresponding books in df_books.

In [6]:
# df_merged.head()

In [7]:
df_books = df_merged[['isbn', 'title', 'author']]
df_books.set_index('isbn', inplace=True)
df_books.head()
#df_books.index

,title,author
isbn,,
034545104X,Flesh Tones: A Novel,M. J. Rose
034545104X,Flesh Tones: A Novel,M. J. Rose
034545104X,Flesh Tones: A Novel,M. J. Rose
034545104X,Flesh Tones: A Novel,M. J. Rose
034545104X,Flesh Tones: A Novel,M. J. Rose


In [8]:
print(df_merged.shape)
df_merged = df_merged.drop(['location', 'age', 'country', 'province',
                   'title', 'author', 'pub_year', 'publisher',
                   'url_s', 'url_m', 'url_l'], axis=1)
print(df_merged.shape)



(379317, 14)
(379317, 3)


In [9]:
gc.collect()

42

In [10]:
import sys

# These are the usual ipython objects, including this one you are creating
ipython_vars = ['In', 'Out', 'exit', 'quit', 'get_ipython', 'ipython_vars']

# Get a sorted list of the objects and their sizes
sorted([(x, sys.getsizeof(globals().get(x))) for x in dir() if not x.startswith('_') and x not in sys.modules and x not in ipython_vars], key=lambda x: x[1], reverse=True)

[('df_books', 84637260),
 ('df_merged', 55236156),
 ('get_clean_data', 136),
 ('get_merged_data_frame', 136),
 ('pd', 80)]

In [11]:
data = df_merged
data.head()
data[(data.user == '2313') & (data.rating == 5)]

,user,isbn,rating
1,2313,034545104X,5
51202,2313,0385482388,5
129253,2313,0399146431,5


## part 2

In [27]:
from surprise import Dataset
from surprise import Reader

from surprise.model_selection import train_test_split
from surprise import accuracy

In [20]:
data_all = Dataset.load_from_df(df = data, reader=Reader(line_format='user item rating'))
data_explicit = Dataset.load_from_df(df = data.loc[data['rating'] != 0],
                                     reader=Reader(line_format='user item rating'))

trainset, testset = train_test_split(data_explicit, test_size=.25)


In [16]:
from surprise import SVD
from surprise.model_selection import cross_validate

# We'll use the famous SVD algorithm.
algo = SVD(n_factors=100, n_epochs=20,)

# Run 5-fold cross-validation and print results
cross_validate(algo, data_all, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    3.4912  3.4899  3.4899  3.4945  3.4933  3.4917  0.0019  
MAE (testset)     2.9136  2.9100  2.9133  2.9083  2.9125  2.9115  0.0021  
Fit time          18.15   18.57   20.96   26.46   27.04   22.24   3.81    
Test time         0.86    0.74    0.84    1.12    0.89    0.89    0.12    


{'test_rmse': array([3.49117796, 3.48986577, 3.48987142, 3.49448083, 3.49331134]),
 'test_mae': array([2.91358134, 2.90997762, 2.91331993, 2.90829784, 2.91248191]),
 'fit_time': (18.152719020843506,
  18.56592845916748,
  20.959733724594116,
  26.45634913444519,
  27.044978618621826),
 'test_time': (0.8568570613861084,
  0.7417552471160889,
  0.8376834392547607,
  1.1173577308654785,
  0.885042667388916)}

In [17]:
cross_validate(algo, data_explicit, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    3.3041  3.2970  3.3037  3.3225  3.3063  3.3067  0.0085  
MAE (testset)     2.9154  2.9122  2.9190  2.9391  2.9264  2.9224  0.0096  
Fit time          7.85    6.96    6.64    6.78    6.77    7.00    0.44    
Test time         0.21    0.18    0.25    0.26    0.25    0.23    0.03    


{'test_rmse': array([3.30409804, 3.29695456, 3.30371155, 3.32248514, 3.3063476 ]),
 'test_mae': array([2.91542615, 2.91215275, 2.91896998, 2.9390833 , 2.92637982]),
 'fit_time': (7.852581262588501,
  6.955885171890259,
  6.640096187591553,
  6.778039455413818,
  6.7660605907440186),
 'test_time': (0.21195316314697266,
  0.18335270881652832,
  0.25226545333862305,
  0.2626931667327881,
  0.2502443790435791)}

In [29]:
# Train the algorithm on the trainset, and predict ratings for the testset
algo.fit(trainset)
predictions = algo.test(testset)

# Then compute RMSE
accuracy.rmse(predictions)



Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 3.3291


TypeError: predict() missing 1 required positional argument: 'iid'

In [33]:
#%% CREATE RATINGS MATRIX
userItemRatingMatrix=pd.pivot_table(data, values='rating',
                                    index=['user'], columns=['isbn'])
df = userItemRatingMatrix.loc[['100004', '100009']]
#df[df.columns[df.apply(lambda s: len(s.unique()) == 1)]]
df[df.columns[df.apply(lambda s: sum(s.isnull()) == 1)]]

isbn,0060392452,0060502258,0060977337,0061015725,0312289871,0312981589,0312982518,0312983654,0345339703,0345339711,...,0553560441,0553582747,059035342X,0671701231,0671742760,0684853949,0743412273,0812543262,1551668246,1853260975
user,,,,,,,,,,,,,,,,,,,,,
100004,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,10.0,0.0,...,NaN,NaN,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100009,8.0,6.0,9.0,NaN,0.0,6.0,8.0,9.0,NaN,NaN,...,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,7.0,9.0


In [36]:
print(algo.predict('100004', '0345339703'))
print(algo.predict('100004', '059035342X'))
print(algo.predict('100009', '0345339703'))
print(algo.predict('100009', '059035342X'))


user: 100004     item: 0345339703 r_ui = None   est = 5.00   {'actual_k': 40, 'was_impossible': False}
user: 100004     item: 059035342X r_ui = None   est = 5.00   {'actual_k': 40, 'was_impossible': False}
user: 100009     item: 0345339703 r_ui = None   est = 5.00   {'actual_k': 9, 'was_impossible': False}
user: 100009     item: 059035342X r_ui = None   est = 5.00   {'actual_k': 23, 'was_impossible': False}


In [25]:
from surprise import KNNBasic
algo = KNNBasic()

cross_validate(algo, data_explicit, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNBasic on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    3.3217  3.3367  3.3103  3.3221  3.3267  3.3235  0.0085  
MAE (testset)     2.9408  2.9482  2.9171  2.9379  2.9383  2.9364  0.0104  
Fit time          1.05    1.08    1.11    1.18    1.09    1.10    0.04    
Test time         0.94    0.77    0.80    0.80    0.76    0.81    0.06    


{'test_rmse': array([3.32173922, 3.33667192, 3.31025058, 3.32207975, 3.32670012]),
 'test_mae': array([2.94077016, 2.94819446, 2.91705266, 2.93785896, 2.93826984]),
 'fit_time': (1.0487525463104248,
  1.0768611431121826,
  1.1072025299072266,
  1.1759490966796875,
  1.0938000679016113),
 'test_time': (0.9361205101013184,
  0.7663702964782715,
  0.8045105934143066,
  0.7992141246795654,
  0.7649576663970947)}

In [ ]:
###############

To process 

we now threshold our data, i.e. only account for data where users have at least 10 ratings (explicit or implicit).  

In [23]:
data.shape

(379317, 3)

In [24]:
#%% CREATE RATINGS MATRIX
userItemRatingMatrix=pd.pivot_table(data, values='rating',
                                    index=['user'], columns=['isbn'])

In [25]:
#type(userItemRatingMatrix)

#userItemRatingMatrix.head()
#userItemRatingMatrix.columns
#2313
#userItemRatingMatrix.iloc[2313]
sum(userItemRatingMatrix.iloc[2313] == 5)

5

**what the hell is this?**

In [41]:
# #%% THRESHOLD CI
# """"""from scipy.stats import sem, t
# from scipy import mean
# confidence = 0.95
# data = ratings_per_isbn['count']

# n = len(data)
# m = mean(data)
# std_err = sem(data)
# h = std_err * t.ppf((1 + confidence) / 2, n - 1)

# start = m - h
# print (start)"""
# #%% VIS ISBN & USER COUNT
# """import seaborn as sns
# ax = sns.distplot(ratings_per_isbn['count'])
# ax2 = ax.twinx()
# sns.boxplot(x=ratings_per_isbn['count'], ax=ax2)
# ax2.set(ylim=(-0.5, 10))"""


In [27]:
import numpy as np
from scipy.spatial.distance import hamming

def distance(user1,user2):
        try:
            user1Ratings = userItemRatingMatrix.transpose()[str(user1)]
            print(len(user1Ratings))
            user2Ratings = userItemRatingMatrix.transpose()[str(user2)]
            distance = hamming(user1Ratings,user2Ratings)
        except: 
            distance = np.NaN
        return distance

def nearestNeighbors(user,K=10):
    allUsers = pd.DataFrame(userItemRatingMatrix.index)
    allUsers = allUsers[allUsers.user!=user]
    allUsers["distance"] = allUsers["user"].apply(lambda x: distance(user,x))
    KnearestUsers = allUsers.sort_values(["distance"],ascending=True)["user"][:K]
    return KnearestUsers

In [32]:
userItemRatingMatrix.shape

(6457, 15441)

In [31]:
allUsers = pd.DataFrame(userItemRatingMatrix.index)

# allUsers = allUsers[allUsers.user!=user]
# allUsers["distance"] = allUsers["user"].apply(lambda x: distance(user,x))
# KnearestUsers = allUsers.sort_values(["distance"],ascending=True)["user"][:K]


# nearestNeighbors(100004)

allUsers.shape

(6457, 1)

In [77]:

print(distance('100004', '100009'))

user1Ratings = userItemRatingMatrix.transpose()[str(user1)]
user2Ratings = userItemRatingMatrix.transpose()[str(user2)]

hamming([1, 0, 0], [0, 1, 0])
#hamming([NaN, 0, 0], [NaN, 1, 0])


15441
1.0


0.6666666666666666

In [58]:
df = userItemRatingMatrix.loc[['100004', '100009']]
df[df.columns[df.apply(lambda s: len(s.unique()) == 1)]]

isbn,0002005018,0002251760,0002259834,0002558122,0006480764,000648302X,0006485200,000649840X,000651202X,0006512062,...,8845906884,8845915611,8878188212,8885989403,9074336329,9074336469,950491036X,9681500830,9681500954,9871138016
user,,,,,,,,,,,,,,,,,,,,,
100004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100009,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [67]:
df[df.columns[df.apply(lambda s: sum(s.isnull()) == 0)]]

isbn
user
100004
100009


In [64]:
sum(df['0060502258'].isnull())

1

In [48]:
#distance()
type(userItemRatingMatrix)


userItemRatingMatrix.index
userItemRatingMatrix.head()


isbn,0002005018,0002251760,0002259834,0002558122,0006480764,000648302X,0006485200,000649840X,000651202X,0006512062,...,8845906884,8845915611,8878188212,8885989403,9074336329,9074336469,950491036X,9681500830,9681500954,9871138016
user,,,,,,,,,,,,,,,,,,,,,
100004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100009,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100053,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100088,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


???

In [82]:
# #%% DEBUGGING
# """NNRatings = userItemRatingMatrix[userItemRatingMatrix.index.isin(KnearestUsers)]
# NNRatings"""
# """avgRating = NNRatings.apply(np.nanmean).dropna()
# avgRating.head()"""
# """booksAlreadyRead = userItemRatingMatrix.transpose()[str(user)].dropna().index
# booksAlreadyRead"""
# """"avgRating = avgRating[~avgRating.index.isin(booksAlreadyRead)]"""


'"avgRating = avgRating[~avgRating.index.isin(booksAlreadyRead)]'

In [25]:
def bookMeta(isbn):
#df_books.set_index('isbn', inplace=True)
#    title = books.at[isbn,"title"]
#    author = books.at[isbn,"author"]
    title = df_books.at[isbn,"title"]
    author = df_books.at[isbn,"author"]
    return title, author

def faveBooks(user,N):
    userRatings = data[data["user"]==user]
    sortedRatings = pd.DataFrame.sort_values(userRatings,['rating'],ascending=[0])[:N] 
    sortedRatings["title"] = sortedRatings["isbn"].apply(bookMeta)
    return sortedRatings

def topN(user,N=3):
    KnearestUsers = nearestNeighbors(user)
    NNRatings = userItemRatingMatrix[userItemRatingMatrix.index.isin(KnearestUsers)]
    avgRating = NNRatings.apply(np.nanmean).dropna()
    booksAlreadyRead = userItemRatingMatrix.transpose()[user].dropna().index
    avgRating = avgRating[~avgRating.index.isin(booksAlreadyRead)]
    topNISBNs = avgRating.sort_values(ascending=False).index[:N]
    return pd.Series(topNISBNs).apply(bookMeta)


In [97]:
#%% DEBUGGING
"""N=3
topNISBNs = avgRating.sort_values(ascending=False).index[:N]
pd.Series(topNISBNs).apply(bookMeta)"""
"""user = '204622'
topN(user)"""


"user = '204622'\ntopN(user)"

In [27]:
topNISBNs = avgRating.sort_values(ascending=False).index[:3]

NameError: name 'avgRating' is not defined

In [98]:
#%% DEBUGGING
"""N=3
topNISBNs = avgRating.sort_values(ascending=False).index[:N]
pd.Series(topNISBNs).apply(bookMeta)"""
"""user = '204622'
topN(user)"""


"user = '204622'\ntopN(user)"

In [100]:
bookMeta('034545104X')

('Flesh Tones: A Novel', 'M. J. Rose')

In [102]:
faveBooks('204622', 5)

,user,isbn,rating,title
8575,204622,0385504209,10,"(The Da Vinci Code, Dan Brown)"
65948,204622,0671027360,10,"(Angels &amp; Demons, Dan Brown)"
169393,204622,0060935464,9,"(To Kill a Mockingbird, Harper Lee)"
499333,204622,1878424114,9,(The Seven Spiritual Laws of Success: A Practi...
484364,204622,0671666258,8,"(American Star, Jackie Collins)"


In [103]:
topN('204622', 5)

C:\Users\user\Anaconda3\envs\recommender\lib\site-packages\pandas\core\apply.py:242: RuntimeWarning: Mean of empty slice
  labels=labels)


0              (Love, Greg &amp; Lauren, Greg Manning)
1    (The Two Towers (The Lord of the Rings, Part 2...
2    (Harry Potter and the Sorcerer's Stone (Book 1...
3                       (Charlotte's Web, E. B. White)
4             (The Secret Life of Bees, Sue Monk Kidd)
Name: isbn, dtype: object